In [125]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

from pandas import read_table

In [6]:
#with open('evaluation_data/R-HSA-1474290/gene_sets.json') as f:
#    data = json.load(f, object_hook=lambda elem: print(elem))
#wv_from_bin = KeyedVectors.load_word2vec_format("__data/nlp/PubMed-Pyysalo-2013/wikipedia-pubmed-and-PMC-w2v.bin", binary=True)  # C binary format
#wv_from_bin["scna"]

In [128]:
read_table("annotation_data/entrezgene2gene_sym.tsv")

In [129]:
immune_cell_data_dir = "raw_data/immune_cells"
generated_immune_marker_genes = read_table(os.path.join(immune_cell_data_dir, "signatures_all.txt"))

#generated_immune_marker_genes <- generated_immune_marker_genes[gene_sym_hsapiens, on=c(gene_symbol="external_gene_name"), nomatch=0]
#used_cell_types <- c("cell", unique(generated_immune_marker_genes$cell_type))


,gene_symbol,original_cell_type,method,cell_type
0,CD28,T cells,mcp_counter,T cell
1,CD3D,T cells,mcp_counter,T cell
2,CD3G,T cells,mcp_counter,T cell
3,CD5,T cells,mcp_counter,T cell
4,CD6,T cells,mcp_counter,T cell
5,CHRM3-AS2,T cells,mcp_counter,T cell
6,CTLA4,T cells,mcp_counter,T cell
7,FLT3LG,T cells,mcp_counter,T cell
8,ICOS,T cells,mcp_counter,T cell
9,MAL,T cells,mcp_counter,T cell
